<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_sept/blob/main/4_pnl/clase_31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Visualizar textos en TensorFlow Projector: Paso a Paso en Google Colab

Este tutorial explica cómo transformar textos (por ejemplo, artículos constitucionales) en vectores para ser visualizados en [https://projector.tensorflow.org](https://projector.tensorflow.org).

---

## 🔹 Paso 1: Cargar tus textos

Simulamos una lista de artículos constitucionales:

```python
textos = [
    "Artículo 1: Chile es una república democrática, fundada en la soberanía popular.",
    "Artículo 2: La soberanía reside en el pueblo de Chile.",
    "Artículo 3: El Estado se organiza en tres poderes independientes.",
    "Artículo 4: La Constitución garantiza la igualdad ante la ley.",
    "Artículo 5: El medio ambiente debe ser protegido por el Estado.",
]
```

---

## 🔹 Paso 2: Instalar librerías necesarias

Usamos `sentence-transformers` para generar buenos embeddings multilingües:

```python
!pip install -q sentence-transformers
```

---

## 🔹 Paso 3: Convertir los textos a vectores (embeddings)

```python
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar modelo multilingüe
modelo = SentenceTransformer('distiluse-base-multilingual-cased')

# Crear los vectores
vectores = modelo.encode(textos)

# Guardamos los textos como etiquetas para visualización
etiquetas = textos
```

---

## 🔹 Paso 4: Guardar archivos `.tsv`

```python
# Guardar los vectores en vectors.tsv
pd.DataFrame(vectores).to_csv("vectors.tsv", sep="\t", header=False, index=False)

# Guardar los textos originales como metadatos
pd.DataFrame(etiquetas).to_csv("metadata.tsv", sep="\t", header=False, index=False)
```

---

## 🔹 Paso 5: Descargar los archivos a tu computador

Puedes usar el explorador de archivos lateral en Colab o ejecutar:

```python
from google.colab import files
files.download("vectors.tsv")
files.download("metadata.tsv")
```

---

## 🔹 Paso 6: Cargar en TensorFlow Projector

1. Ir a: [https://projector.tensorflow.org/](https://projector.tensorflow.org/)
2. Hacer clic en “**Load**”
3. Subir:
   - `vectors.tsv` como **Tensor**
   - `metadata.tsv` como **Metadata**
4. Explorar con **PCA, t-SNE o UMAP**, y buscar similitudes entre textos.

---

## ✅ Opcional: Leer textos desde archivo `.txt` o `.csv`

Si tienes los artículos en un archivo `constitucion.txt` (una línea por artículo), haz:

```python
# Subir archivo
from google.colab import files
uploaded = files.upload()

# Leer textos desde el archivo
with open("constitucion.txt", encoding="utf-8") as f:
    textos = [line.strip() for line in f if line.strip()]
```

---

## 🎉 ¡Listo!
Ahora puedes explorar tus textos en un espacio semántico con TensorFlow Projector.


In [1]:
import os
import shutil

# Crear la carpeta 'corpus' si no existe
os.makedirs("corpus", exist_ok=True)

# Mover archivos PDF y EPUB cargados al entorno a la carpeta 'corpus'
for file in os.listdir():
    if file.endswith(".pdf") or file.endswith(".epub"):
        print(f"Moviendo {file} a carpeta corpus")
        shutil.move(file, os.path.join("corpus", file))

# Confirmar contenido de la carpeta
print("\nArchivos en la carpeta 'corpus':")
print(os.listdir("corpus"))



Archivos en la carpeta 'corpus':
[]


In [3]:
from google.colab import files

# Subir archivos .pdf y .epub desde tu equipo
uploaded = files.upload()

Saving gm_tomo2.epub to gm_tomo2.epub
Saving tomo3_e.epub to tomo3_e.epub
Saving tomo4_gm.epub to tomo4_gm.epub
Saving tomo5_gm(2).epub to tomo5_gm(2).epub
Saving tomo6_m-1.pdf to tomo6_m-1.pdf
Saving tomo7_m.pdf to tomo7_m.pdf
Saving tomo8_m.pdf to tomo8_m.pdf


In [4]:
%%capture
!pip install ebooklib PyMuPDF

In [5]:
import os
import shutil

# Crear carpeta corpus si no existe
os.makedirs("corpus", exist_ok=True)

# Mover todos los archivos .pdf y .epub que estén en la raíz a la carpeta corpus
for archivo in os.listdir():
    if archivo.endswith(".pdf") or archivo.endswith(".epub"):
        if not archivo.startswith("corpus"):  # Evitar mover de nuevo si ya están ahí
            print(f"Moviendo {archivo} a carpeta corpus/")
            shutil.move(archivo, os.path.join("corpus", archivo))

# Verificación
print("\nContenido de la carpeta corpus:")
print(os.listdir("corpus"))

Moviendo tomo6_m-1.pdf a carpeta corpus/
Moviendo tomo5_gm(2).epub a carpeta corpus/
Moviendo tomo4_gm.epub a carpeta corpus/
Moviendo tomo3_e.epub a carpeta corpus/
Moviendo tomo8_m.pdf a carpeta corpus/
Moviendo gm_tomo1.epub a carpeta corpus/
Moviendo gm_tomo2.epub a carpeta corpus/
Moviendo tomo7_m.pdf a carpeta corpus/

Contenido de la carpeta corpus:
['tomo6_m-1.pdf', 'tomo5_gm(2).epub', 'tomo4_gm.epub', 'tomo3_e.epub', 'tomo8_m.pdf', 'gm_tomo1.epub', 'gm_tomo2.epub', 'tomo7_m.pdf']


In [6]:
# 1. Librerías necesarias
import os
import fitz  # PyMuPDF
from ebooklib import epub, ITEM_DOCUMENT
from bs4 import BeautifulSoup

def extract_epub_text(path):
    book = epub.read_epub(path)
    text = ''
    for item in book.get_items():
        if item.get_type() == ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_content(), 'html.parser')
            text += soup.get_text()
    return text


# 3. Función para PDF
def extract_pdf_text(path):
    text = ''
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# 4. Extraer textos
corpus_path = "corpus"
textos_completos = ""

for archivo in os.listdir(corpus_path):
    ruta = os.path.join(corpus_path, archivo)
    if archivo.endswith(".epub"):
        print(f"Extrayendo EPUB: {archivo}")
        textos_completos += extract_epub_text(ruta) + "\n"
    elif archivo.endswith(".pdf"):
        print(f"Extrayendo PDF: {archivo}")
        textos_completos += extract_pdf_text(ruta) + "\n"

print("\nLongitud total del corpus:", len(textos_completos), "caracteres")

# 5. Guardar en archivo
with open("gabriela_mistral_corpus.txt", "w", encoding="utf-8") as f:
    f.write(textos_completos)

Extrayendo PDF: tomo6_m-1.pdf
Extrayendo EPUB: tomo5_gm(2).epub
Extrayendo EPUB: tomo4_gm.epub
Extrayendo EPUB: tomo3_e.epub
Extrayendo PDF: tomo8_m.pdf
Extrayendo EPUB: gm_tomo1.epub
Extrayendo EPUB: gm_tomo2.epub
Extrayendo PDF: tomo7_m.pdf

Longitud total del corpus: 4647275 caracteres


In [7]:
# Cargar corpus
with open("gabriela_mistral_corpus.txt", "r", encoding="utf-8") as f:
    corpus = f.read()

# Limpiar y tokenizar
import re
tokens = re.findall(r'\b\w+\b', corpus.lower())  # palabras minúsculas sin signos
print(f"Número de tokens: {len(tokens)}")

Número de tokens: 840658


In [8]:
!pip install -q sentence-transformers

In [9]:
# Supongamos que etiquetas es un solo string largo
with open("gabriela_mistral_corpus.txt", encoding="utf-8") as f:
    texto_largo = f.read()

# O si ya lo tienes cargado:
# texto_largo = etiquetas

# Dividir por líneas con contenido
etiquetas = [line.strip() for line in texto_largo.splitlines() if line.strip()]

In [10]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar modelo multilingüe
modelo = SentenceTransformer('distiluse-base-multilingual-cased')

# Crear los vectores
vectores = modelo.encode(corpus)

# Guardamos los textos como etiquetas para visualización
etiquetas = corpus

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [12]:
print(len(etiquetas))
print(etiquetas[:5])

4647275
S E L


In [14]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar el texto
with open("gabriela_mistral_corpus.txt", "r", encoding="utf-8") as f:
    frases = [line.strip() for line in f if line.strip()]

# Generar embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
vectors = model.encode(frases)

# Guardar archivos para TensorFlow Projector
pd.DataFrame(vectors).to_csv("vectors.tsv", sep="\t", header=False, index=False)
pd.DataFrame({'text': frases}).to_csv("metadata.tsv", sep="\t", header=False, index=False)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from google.colab import files
files.download("vectors.tsv")
files.download("metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# ============================================================
# 1. Importar librerías necesarias
# ============================================================
import re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# ============================================================
# 2. Cargar el corpus de Gabriela Mistral
# ============================================================
with open("gabriela_mistral_corpus.txt", "r", encoding="utf-8") as f:
    texto = f.read()

# ============================================================
# 3. Preprocesamiento del texto
#    - Eliminar números
#    - Eliminar caracteres no alfabéticos
#    - Convertir a minúsculas
# ============================================================
texto = re.sub(r"\d+", "", texto)                # eliminar números
texto = re.sub(r"[^A-Za-zÁÉÍÓÚáéíóúÑñüÜ\s]", "", texto)  # eliminar signos
texto = texto.lower()                            # pasar a minúsculas

# ============================================================
# 4. Tokenización por palabra (no por frase)
# ============================================================
# Dividir en palabras (puedes cambiarlo por split() o usar nltk.word_tokenize)
palabras = texto.split()
# Remover duplicados opcionalmente
palabras_unicas = list(set(palabras))

print(f"Total de palabras únicas: {len(palabras_unicas)}")

# ============================================================
# 5. Generar embeddings por palabra
# ============================================================
modelo = SentenceTransformer("all-MiniLM-L6-v2")
vectores = modelo.encode(palabras_unicas)

# ============================================================
# 6. Guardar archivos para TensorFlow Projector
# ============================================================
# Cada fila del TSV de vectores corresponde a una palabra
pd.DataFrame(vectores).to_csv("vectors.tsv", sep="\t", header=False, index=False)

# El archivo de metadata contiene las etiquetas (las palabras)
pd.DataFrame({'word': palabras_unicas}).to_csv("metadata.tsv", sep="\t", header=False, index=False)

print("Listo. Archivos 'vectors.tsv' y 'metadata.tsv' generados correctamente.")

files.download("vectors.tsv")
files.download("metadata.tsv")


Total de palabras únicas: 51322
Listo. Archivos 'vectors.tsv' y 'metadata.tsv' generados correctamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# https://www.kaggle.com/datasets/mauridb/product-data-from-walmart-usa-with-embeddings?utm_source=chatgpt.com

### **1. Instalación e importación de librerías**

```python
!pip install sentence-transformers faiss-cpu gradio kagglehub -q
```

Instala las librerías necesarias:

* **sentence-transformers:** genera representaciones vectoriales (embeddings) de texto.
* **faiss-cpu:** permite crear un índice para realizar búsquedas vectoriales eficientes.
* **gradio:** crea una interfaz web interactiva para el usuario.
* **kagglehub:** descarga datasets directamente desde Kaggle.

```python
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import gradio as gr
import os
```

Importa las librerías ya instaladas:

* `pandas` para manejo de datos tabulares.
* `numpy` para operaciones numéricas.
* `SentenceTransformer` para crear embeddings.
* `faiss` para la búsqueda vectorial.
* `gradio` para crear la interfaz.
* `os` para trabajar con rutas de archivos.

---

### **2. Cargar dataset desde KaggleHub**

```python
import kagglehub
```

Importa el módulo `kagglehub` para conectar con Kaggle.

```python
path = kagglehub.dataset_download("mauridb/product-data-from-walmart-usa-with-embeddings")
print("Ruta de los archivos del dataset:", path)
```

Descarga el dataset **“product-data-from-walmart-usa-with-embeddings”** desde Kaggle y guarda su ruta local. Luego la muestra por pantalla.

```python
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
print("Archivos encontrados:", csv_files)
```

Busca dentro de la carpeta descargada todos los archivos con extensión `.csv`.
El resultado se imprime en consola para confirmar cuál archivo se usará.

```python
df = pd.read_csv(os.path.join(path, csv_files[0]))
print("Columnas disponibles:", df.columns.tolist())
```

Carga el primer archivo CSV encontrado en un DataFrame de pandas (`df`).
Luego imprime la lista de columnas del dataset para inspección.

---

### **3. Limpieza y selección de campos**

```python
df["text"] = (
    df["product_name"].fillna("") + ". " +
    df["description"].fillna("") + ". " +
    df["category"].fillna("") + ". " +
    df["brand"].fillna("")
)
```

Crea una nueva columna llamada `text` combinando información relevante:

* nombre del producto,
* descripción,
* categoría,
* marca.

Cada campo se une en una sola cadena, separada por puntos, y se reemplazan valores nulos con cadenas vacías.

```python
df = df[df["text"].str.strip() != ""]
```

Elimina filas cuyo texto combinado esté vacío o solo tenga espacios.

```python
df[["product_name", "category", "brand", "text"]].head(2)
```

Muestra las primeras dos filas con las columnas más importantes para verificar que la combinación se realizó correctamente.

---

### **4. Generar embeddings semánticos**

```python
modelo = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
```

Carga un modelo multilingüe preentrenado de **Sentence Transformers**, capaz de transformar texto en vectores numéricos que capturan su significado semántico.

```python
embeddings = modelo.encode(df["text"].tolist(), show_progress_bar=True)
```

Convierte la columna `text` en una lista y genera un embedding para cada producto.
El parámetro `show_progress_bar=True` muestra una barra de progreso durante el proceso.

---

### **5. Crear índice FAISS**

```python
dimension = embeddings.shape[1]
```

Obtiene la dimensión de los vectores generados (por ejemplo, 768).

```python
index = faiss.IndexFlatL2(dimension)
```

Crea un índice **FAISS** basado en la distancia euclidiana (L2). Este índice permitirá buscar los productos más similares a una consulta dada.

```python
index.add(np.array(embeddings, dtype=np.float32))
```

Agrega todos los embeddings al índice, convirtiéndolos a tipo `float32` (formato requerido por FAISS).

---

### **6. Función de búsqueda**

```python
def buscar(query, k=5):
    query_vec = modelo.encode([query])
```

Define una función llamada `buscar`.
Primero convierte el texto de consulta (`query`) en su vector correspondiente.

```python
    D, I = index.search(np.array(query_vec, dtype=np.float32), k)
```

Realiza la búsqueda en el índice FAISS, retornando:

* `D`: las distancias (menor = más parecido).
* `I`: los índices de los productos más cercanos.
  Busca los `k` productos más similares (por defecto 5).

```python
    resultados = df.iloc[I[0]][["product_name", "category", "brand", "description"]]
    resultados["distancia"] = D[0]
    return resultados
```

Selecciona del DataFrame los productos correspondientes a los índices encontrados y agrega su distancia de similitud.
Devuelve una tabla con los resultados más relevantes.

---

### **7. Interfaz interactiva con Gradio**

```python
def interfaz(query):
    resultados = buscar(query, k=5)
```

Crea una segunda función que servirá de interfaz.
Cada vez que el usuario escriba una consulta, se llama a la función `buscar`.

```python
    texto = ""
    for _, fila in resultados.iterrows():
        texto += f"### {fila['product_name']} ({fila['category']})\n"
        texto += f"Marca: {fila['brand']}\n"
        texto += f"Distancia: {fila['distancia']:.4f}\n\n"
        texto += f"{fila['description']}\n\n---\n"
    return texto
```

Construye un texto en formato Markdown para mostrar en pantalla:

* el nombre del producto y su categoría,
* la marca,
* la distancia (nivel de similitud),
* la descripción.

El separador `---` se usa para dividir visualmente los resultados.

```python
demo = gr.Interface(
    fn=interfaz,
    inputs=gr.Textbox(label="Consulta de producto", placeholder="Ejemplo: snack saludable sin azúcar"),
    outputs="markdown",
    title="Buscador Semántico Walmart",
    description="Busca productos de Walmart por significado, combinando nombre, descripción, categoría y marca."
)
```

Crea la interfaz de **Gradio**:

* `fn=interfaz`: función que se ejecuta al enviar la consulta.
* `inputs`: una caja de texto donde el usuario escribe la búsqueda.
* `outputs`: muestra los resultados formateados en Markdown.
* `title` y `description`: definen el encabezado y la descripción visibles en la interfaz.

```python
demo.launch()
```

Ejecuta la aplicación localmente o en Google Colab.
Abre una URL donde el usuario puede probar el buscador escribiendo cualquier descripción de producto y recibiendo resultados semánticamente similares.


In [ ]:
# ============================================================
# 1. Instalación e importación de librerías
# ============================================================
!pip install sentence-transformers faiss-cpu gradio kagglehub -q

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import gradio as gr
import os

# ============================================================
# 2. Cargar dataset desde KaggleHub
# ============================================================
import kagglehub

path = kagglehub.dataset_download("mauridb/product-data-from-walmart-usa-with-embeddings")
print("Ruta de los archivos del dataset:", path)

csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
print("Archivos encontrados:", csv_files)

df = pd.read_csv(os.path.join(path, csv_files[0]))
print("Columnas disponibles:", df.columns.tolist())

# ============================================================
# 3. Limpieza y selección de campos
# ============================================================
# Combina información textual relevante (nombre + descripción + categoría + marca)
df["text"] = (
    df["product_name"].fillna("") + ". " +
    df["description"].fillna("") + ". " +
    df["category"].fillna("") + ". " +
    df["brand"].fillna("")
)

df = df[df["text"].str.strip() != ""]

# Muestra un ejemplo
df[["product_name", "category", "brand", "text"]].head(2)

# ============================================================
# 4. Generar embeddings semánticos
# ============================================================
modelo = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
embeddings = modelo.encode(df["text"].tolist(), show_progress_bar=True)

# ============================================================
# 5. Crear índice FAISS
# ============================================================
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings, dtype=np.float32))

# ============================================================
# 6. Función de búsqueda
# ============================================================
def buscar(query, k=5):
    query_vec = modelo.encode([query])
    D, I = index.search(np.array(query_vec, dtype=np.float32), k)
    resultados = df.iloc[I[0]][["product_name", "category", "brand", "description"]]
    resultados["distancia"] = D[0]
    return resultados

# ============================================================
# 7. Interfaz interactiva con Gradio
# ============================================================
def interfaz(query):
    resultados = buscar(query, k=5)
    texto = ""
    for _, fila in resultados.iterrows():
        texto += f"### {fila['product_name']} ({fila['category']})\n"
        texto += f"Marca: {fila['brand']}\n"
        texto += f"Distancia: {fila['distancia']:.4f}\n\n"
        texto += f"{fila['description']}\n\n---\n"
    return texto

demo = gr.Interface(
    fn=interfaz,
    inputs=gr.Textbox(label="Consulta de producto", placeholder="Ejemplo: snack saludable sin azúcar"),
    outputs="markdown",
    title="Buscador Semántico Walmart",
    description="Busca productos de Walmart por significado, combinando nombre, descripción, categoría y marca."
)

demo.launch()


Using Colab cache for faster access to the 'product-data-from-walmart-usa-with-embeddings' dataset.
Ruta de los archivos del dataset: /kaggle/input/product-data-from-walmart-usa-with-embeddings
Archivos encontrados: ['walmart-product-with-embeddings-dataset-usa.csv']
Columnas disponibles: ['id', 'source_unique_id', 'crawl_timestamp', 'product_url', 'product_name', 'description', 'list_price', 'sale_price', 'brand', 'item_number', 'gtin', 'package_size', 'category', 'postal_code', 'available', 'embedding']


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/938 [00:00<?, ?it/s]

## **1. Cómo FAISS calcula las similitudes**

**FAISS (Facebook AI Similarity Search)** es una biblioteca desarrollada por Meta para realizar búsquedas de similitud en grandes volúmenes de vectores.
Cuando tú generas embeddings con *SentenceTransformer*, cada texto se transforma en un vector de cientos de dimensiones (por ejemplo, 768).
FAISS se encarga de comparar esos vectores entre sí de forma eficiente.

Veamos cómo ocurre esto paso a paso:

1. **Construcción del índice**

   ```python
   index = faiss.IndexFlatL2(dimension)
   ```

   Aquí se crea un índice “plano” (`FlatL2`), que usa la **distancia euclidiana al cuadrado (L2)** para medir la similitud entre vectores.
   Cuanto **menor** es la distancia L2 entre dos vectores, más similares son sus significados.

2. **Agregado de vectores**

   ```python
   index.add(np.array(embeddings, dtype=np.float32))
   ```

   Todos los vectores (uno por producto) se almacenan en el índice.
   FAISS optimiza internamente la estructura para hacer búsquedas rápidas incluso con miles de vectores.

3. **Consulta**

   ```python
   D, I = index.search(np.array(query_vec, dtype=np.float32), k)
   ```

   * **`query_vec`**: es el embedding del texto de búsqueda.
   * **`D`**: es un arreglo con las distancias entre la consulta y los productos más cercanos.
   * **`I`**: contiene los índices (posiciones) de los productos más similares dentro del DataFrame original.

   Por ejemplo, si `D[0][0] = 0.15` y `D[0][4] = 0.88`, el primer resultado es mucho más parecido al texto buscado que el quinto.

4. **Interpretación**
   En un índice L2, la similitud se basa en **distancia mínima**:

   * Distancia 0: vectores idénticos.
   * Distancia pequeña: textos muy relacionados.
   * Distancia grande: textos sin relación semántica.

Si quisieras usar similitud **coseno** en lugar de L2 (más común en procesamiento de texto), podrías normalizar los vectores antes de añadirlos al índice:

```python
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
index = faiss.IndexFlatIP(dimension)  # IP = Inner Product
```

De este modo, FAISS buscaría los vectores con el **producto interno más alto**, equivalente a mayor similitud coseno.

---

## **2. Cómo usar los embeddings ya incluidos en el dataset**

El dataset de Walmart que descargaste **ya contiene una columna llamada `embedding`**, que parece guardar los vectores precomputados.
Esto significa que **no necesitas volver a generar embeddings** con `SentenceTransformer`, lo que puede ahorrar tiempo y recursos.

Sin embargo, esos embeddings vienen almacenados como texto (por ejemplo: `"[0.123, -0.452, 0.876, ...]"`), por lo que primero hay que convertirlos a arreglos numéricos de `numpy`.

Aquí tienes cómo hacerlo:

```python
import ast

# Convierte los embeddings desde texto a lista numérica
df["embedding"] = df["embedding"].apply(ast.literal_eval)

# Crea una matriz de numpy con los vectores
embeddings = np.array(df["embedding"].tolist()).astype("float32")

# Crea el índice FAISS directamente con ellos
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Función de búsqueda adaptada
def buscar(query, k=5):
    query_vec = modelo.encode([query])  # aún necesitas el modelo para consultas nuevas
    D, I = index.search(np.array(query_vec, dtype=np.float32), k)
    resultados = df.iloc[I[0]][["product_name", "category", "brand", "description"]]
    resultados["distancia"] = D[0]
    return resultados
```

En este caso:

* Usas los embeddings del dataset (ya calculados para cada producto).
* Solo generas un embedding nuevo cuando el usuario realiza una consulta.
* El tiempo de procesamiento se reduce considerablemente.

---

## **Resumen conceptual**

| Proceso                                        | Qué hace                                                                     | Cuándo usarlo                                                         |
| ---------------------------------------------- | ---------------------------------------------------------------------------- | --------------------------------------------------------------------- |
| **Generar embeddings con SentenceTransformer** | Convierte texto en vectores semánticos nuevos.                               | Si el dataset no tiene embeddings o si quieres usar tu propio modelo. |
| **Usar embeddings preexistentes**              | Carga los vectores ya calculados en el dataset.                              | Si el dataset ya incluye una columna `embedding`.                     |
| **Índice FAISS (L2 o coseno)**                 | Permite buscar rápidamente los textos más similares en el espacio vectorial. | Siempre necesario para la búsqueda semántica eficiente.               |


In [ ]:
import pandas as pd
import numpy as np
import ast
from google.colab import files

# ============================================================
# 1. Cargar dataset desde KaggleHub (si aún no está cargado)
# ============================================================
import kagglehub

path = kagglehub.dataset_download("mauridb/product-data-from-walmart-usa-with-embeddings")
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
df = pd.read_csv(os.path.join(path, csv_files[0]))

# ============================================================
# 2. Convertir embeddings desde texto a listas numéricas
# ============================================================
# Convierte la columna 'embedding' (tipo str) a lista de floats
df["embedding"] = df["embedding"].apply(ast.literal_eval)

# ============================================================
# 3. Crear el archivo vectors.tsv
# ============================================================
# Cada fila del TSV será un vector de embedding
vectors = np.array(df["embedding"].tolist())
np.savetxt("vectors.tsv", vectors, delimiter="\t")

# ============================================================
# 4. Crear el archivo metadata.tsv
# ============================================================
# Incluye campos descriptivos para identificar los puntos
metadata = df[["product_name", "category", "brand", "sale_price"]].fillna("")
metadata.to_csv("metadata.tsv", sep="\t", index=False)

# ============================================================
# 5. Descargar los archivos para subirlos al TensorFlow Projector
# ============================================================
files.download("vectors.tsv")
files.download("metadata.tsv")
